In [2]:
from zipline.data.bundles.core import load
import os
from dotenv import load_dotenv

load_dotenv()  # loads environment variables from .env file if used

bundle_data = load("quandl", os.environ)


ModuleNotFoundError: No module named 'zipline'

In [ ]:
import pandas as pd
import numpy as np
from zipline import run_algorithm
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import AverageDollarVolume, CustomFactor, Returns
from zipline.finance import commission, slippage
from zipline.pipeline.filters import StaticAssets, CustomFilter
from zipline.api import (
    attach_pipeline,
    calendars,
    pipeline_output,
    date_rules,
    time_rules,
    set_commission,
    set_slippage,
    record,
    order_target_percent,
    get_open_orders,
    get_datetime,
    schedule_function,
    symbol
)
import pandas_datareader as web

In [ ]:
class VolRegimeFilter(CustomFilter):
    inputs = [Returns(window_length=2)]
    window_length = 126
    def compute(self, today, assets, out, returns):
        volatility = np.std(returns, axis=0)
        ranks = pd.Series(volatility).rank(pct=True)
        out[:] = ranks > 0.5

class MomReversalFilter(CustomFilter):
    inputs = [USEquityPricing.close]
    window_length = 126
    def compute(self, today, assets, out, close):
        zscores = (close[-1] - np.mean(close, axis=0)) / np.std(close, axis=0)
        out[:] = zscores < 0.5

class MRComponent(CustomFactor):
    inputs = [USEquityPricing.close, Returns(window_length=22)]
    window_length = 25
    def compute(self, today, assets, out, close, returns):
        price_change = (close[-1] - close[-4]) / close[-4]
        vol = np.std(returns[-22:], axis=0)
        out[:] = -pd.Series(price_change / vol).rank(pct=True)

class VolComponent(CustomFactor):
    inputs = [USEquityPricing.volume, AverageDollarVolume(window_length=20)]
    window_length = 1
    def compute(self, today, assets, out, volume, adv20):
        out[:] = pd.Series(volume[-1] / adv20[-1]).rank(pct=True)

def make_pipeline():

    SNP_500 = [
    'A', 'AAL', 'AAPL', 'ABBV', 'ABNB', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM',
    'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AJG', 'AKAM',
    'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN',
    'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'AON', 'AOS', 'APA', 'APD', 'APH',
    'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXON', 'AXP',
    'AZO', 'BA', 'BAC', 'BALL', 'BAX', 'BBWI', 'BBY', 'BDX', 'BEN', 'BF.B',
    'BG', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLDR', 'BLK', 'BMY', 'BR',
    'BRO', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB',
    'CBOE', 'CBRE', 'CCI', 'CCL', 'CDAY', 'CDNS', 'CDW', 'CE', 'CEG', 'CF',
    'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA',
    'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COO', 'COP', 'COR',
    'COST', 'CPB', 'CPRT', 'CPT', 'CRL', 'CRM', 'CSCO', 'CSGP', 'CSX', 'CTAS',
    'CTLT', 'CTRA', 'CTSH', 'CTVA', 'CVS', 'CVX', 'CZR', 'D', 'DAL', 'DD',
    'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCK', 'DISH', 'DLR',
    'DLTR', 'DOV', 'DOW', 'DPZ', 'DRI', 'DTE', 'DUK', 'DVA', 'DVN', 'DXCM',
    'DXC', 'EA', 'EBAY', 'ECL', 'ED', 'EFX', 'EG', 'EIX', 'EL', 'ELV', 'EMN',
    'EMR', 'ENPH', 'EOG', 'EPAM', 'EQIX', 'EQR', 'EQT', 'ES', 'ESS', 'ETN',
    'ETR', 'ETSY', 'EVRG', 'EW', 'EXC', 'EXPD', 'EXPE', 'EXR', 'F', 'FANG',
    'FAST', 'FCX', 'FDS', 'FDX', 'FE', 'FFIV', 'FICO', 'FIS', 'FISV', 'FITB',
    'FLT', 'FMC', 'FOX', 'FOXA', 'FRC', 'FRT', 'FSLR', 'FTNT', 'FTV', 'GD',
    'GE', 'GEN', 'GILD', 'GIS', 'GL', 'GLW', 'GM', 'GNRC', 'GOOG', 'GOOGL',
    'GPC', 'GPN', 'GRMN', 'GS', 'GWW', 'HAL', 'HAS', 'HBAN', 'HCA', 'HD',
    'HES', 'HIG', 'HII', 'HLT', 'HOLX', 'HON', 'HPE', 'HPQ', 'HRL', 'HSIC',
    'HST', 'HSY', 'HUM', 'HWM', 'IBM', 'ICE', 'IDXX', 'IEX', 'IFF', 'ILMN',
    'INCY', 'INTC', 'INTU', 'INVH', 'IP', 'IPG', 'IQV', 'IR', 'IRM', 'ISRG',
    'IT', 'ITW', 'IVZ', 'J', 'JBHT', 'JBL', 'JCI', 'JKHY', 'JNJ', 'JNPR',
    'JPM', 'K', 'KDP', 'KEY', 'KEYS', 'KIM', 'KLAC', 'KMB', 'KMI', 'KMX',
    'KO', 'KR', 'KRC', 'KSS', 'L', 'LDOS', 'LEN', 'LH', 'LHX', 'LIN', 'LKQ',
    'LLY', 'LMT', 'LNC', 'LNT', 'LOW', 'LRCX', 'LUMN', 'LUV', 'LW', 'LYB',
    'LYV', 'M', 'MA', 'MAA', 'MAR', 'MAS', 'MCD', 'MCHP', 'MCK', 'MCO',
    'MDLZ', 'MDT', 'MET', 'META', 'MGM', 'MHK', 'MKC', 'MKTX', 'MLM', 'MMC',
    'MMM', 'MNST', 'MO', 'MOS', 'MPC', 'MPWR', 'MRK', 'MRNA', 'MRO', 'MS',
    'MSCI', 'MSFT', 'MSI', 'MTB', 'MTCH', 'MTD', 'MU', 'NCLH', 'NDAQ', 'NDSN',
    'NEE', 'NEM', 'NFLX', 'NI', 'NKE', 'NOC', 'NOW', 'NRG', 'NSC', 'NTAP',
    'NTRS', 'NUE', 'NVDA', 'NVR', 'NWS', 'NWSA', 'NXPI', 'O', 'ODFL', 'OGN',
    'OKE', 'OMC', 'ON', 'ORCL', 'ORLY', 'OTIS', 'OXY', 'PARA', 'PAYC', 'PAYX',
    'PCAR', 'PCG', 'PEAK', 'PEG', 'PEP', 'PFE', 'PFG', 'PG', 'PGR', 'PH',
    'PHM', 'PKG', 'PLD', 'PM', 'PNC', 'PNR', 'PNW', 'POOL', 'PPG', 'PPL',
    'PRU', 'PSA', 'PSX', 'PTC', 'PWR', 'PXD', 'PYPL', 'QRVO', 'RCL', 'RE',
    'REG', 'REGN', 'RF', 'RHI', 'RJF', 'RL', 'RMD', 'ROK', 'ROL', 'ROP',
    'ROST', 'RSG', 'RTX', 'SBAC', 'SBNY', 'SBUX', 'SCHW', 'SEDG', 'SEE',
    'SHW', 'SIVB', 'SJM', 'SLB', 'SLG', 'SNA', 'SNPS', 'SO', 'SPG', 'SPGI',
    'SRE', 'STE', 'STLD', 'STT', 'STX', 'STZ', 'SWK', 'SWKS', 'SYF', 'SYK',
    'SYY', 'T', 'TAP', 'TDG', 'TDY', 'TECH', 'TEL', 'TER', 'TFC', 'TFX',
    'TGT', 'TJX', 'TMO', 'TMUS', 'TPR', 'TRGP', 'TRMB', 'TROW', 'TRV', 'TSCO',
    'TSLA', 'TSN', 'TT', 'TTWO', 'TWTR', 'TXN', 'TXT', 'TYL', 'UAL', 'UDR',
    'UHS', 'ULTA', 'UNH', 'UNP', 'UPS', 'URI', 'USB', 'V', 'VFC', 'VICI',
    'VLO', 'VMC', 'VNO', 'VRSK', 'VRSN', 'VRTX', 'VTR', 'VTRS', 'VZ', 'WAB',
    'WAT', 'WBA', 'WBD', 'WDC', 'WEC', 'WELL', 'WFC', 'WHR', 'WM', 'WMB',
    'WMT', 'WRB', 'WRK', 'WST', 'WTW', 'WY', 'WYNN', 'XEL', 'XOM', 'XRAY',
    'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZION', 'ZTS'
    ]
    valid_assets = []
    for ticker in SNP_500:
        try:
            valid_assets.append(symbol(ticker))
        except Exception:
            pass
    universe = StaticAssets(valid_assets)

    alpha = 0.5 * MRComponent() + 0.5 * VolComponent()
    trade_condition = VolRegimeFilter() & MomReversalFilter()

    return Pipeline(
        columns={
            'alpha': alpha,
            'trade_condition': trade_condition,
        },
        screen=universe & trade_condition
    )

# Called before each trading day to get the latest pipeline output
def before_trading_start(context, data):
    # Store pipeline results in context for use in rebalancing
    context.factor_data = pipeline_output("factor_pipeline")

# Initialize the algorithm and set up schedules, commission, and slippage
def initialize(context):
    # Attach the pipeline to the algorithm
    attach_pipeline(make_pipeline(), "factor_pipeline")
    # Schedule weekly rebalancing at market open
    schedule_function(
        rebalance,
        date_rules.every_day(),
        time_rules.market_open(),
        calendar=calendars.US_EQUITIES,
    )
    # Set commission model (per share)
    set_commission(
        us_equities=commission.PerShare(cost=0.00075, min_trade_cost=0.01)
    )
    # Set slippage model (volume share based)
    set_slippage(
        us_equities=slippage.VolumeShareSlippage(volume_limit=0.0025, price_impact=0.01)
    )

def rebalance(context, data):
    factor_data = context.factor_data  # Pipeline output (DataFrame)
    alpha = factor_data['alpha']      # Series: index=assets, values=alpha
    assets = factor_data.index

    # Split into longs (alpha > 0) and shorts (alpha < 0)
    longs = alpha[alpha > 0]
    shorts = alpha[alpha < 0]

    # Normalize weights: sum of long weights = +1, sum of short weights = -1
    long_total = longs.sum()
    short_total = shorts.sum()  # This is negative

    long_weights = longs / long_total if long_total != 0 else longs * 0
    short_weights = shorts / abs(short_total) if short_total != 0 else shorts * 0

    # Assets to fully exit (not in new long/short lists)
    divest = set(context.portfolio.positions.keys()) - set(longs.index.union(shorts.index))
    print(f"{get_datetime().date()} | Longs {len(longs)} | Shorts {len(shorts)} | {context.portfolio.portfolio_value}")

    exec_trades(data, assets=divest, weights=None, target_percent=0)

    # Place proportional orders for longs
    exec_trades(data, assets=long_weights.index, weights=long_weights, target_percent=None)
    # Place proportional orders for shorts (negative weights)
    exec_trades(data, assets=short_weights.index, weights=short_weights, target_percent=None, is_short=True)

    # Optional: record diagnostics
    record(num_longs=len(longs), num_shorts=len(shorts), portfolio_value=context.portfolio.portfolio_value)

def exec_trades(data, assets, weights=None, target_percent=None, is_short=False):
    for asset in assets:
        if data.can_trade(asset) and not get_open_orders(asset):
            if weights is not None:
                # Use proportional weight (negative for shorts)
                weight = -weights[asset] if is_short else weights[asset]
                order_target_percent(asset, weight)
            else:
                # Use fixed target_percent (for divest)
                order_target_percent(asset, target_percent)


# Plot portfolio value at the end of the backtest
def analyze(context, perf):
    perf.portfolio_value.plot()

# Set up backtest parameters
start = pd.Timestamp("2016")
end = pd.Timestamp("2018")
# Get S&P 500 index data from FRED for benchmark returns
sp500 = web.DataReader('SP500', 'fred', start, end).SP500
benchmark_returns = sp500.pct_change()

# Run the backtest with all the above settings
perf = run_algorithm(
    start=start,
    end=end,
    initialize=initialize,
    analyze=analyze,
    benchmark_returns=benchmark_returns,
    before_trading_start=before_trading_start,
    capital_base=100_000,
    bundle="quandl"
)
# Save results to a file for later analysis
perf.to_pickle("mean_reversion.pickle")


C:\Users\nickt\AppData\Local\Temp\tmpo4xehaa8\ipykernel_18296\1962184903.py:180: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  benchmark_returns = sp500.pct_change()
C:\Users\nickt\AppData\Local\Temp\tmpo4xehaa8\ipykernel_18296\1962184903.py:28: RuntimeWarning: invalid value encountered in divide
  out[:] = pd.Series(volume[-1] / adv20[-1]).rank(pct=True)


2016-01-04 | Longs 13 | Shorts 50 | 100000.0
2016-01-05 | Longs 12 | Shorts 54 | 99993.72853254508
2016-01-06 | Longs 17 | Shorts 50 | 92112.15722550495
2016-01-07 | Longs 31 | Shorts 34 | 85544.67662576381
2016-01-08 | Longs 30 | Shorts 38 | 82996.6065952266
2016-01-11 | Longs 32 | Shorts 40 | 78183.88784176638


C:\Users\nickt\AppData\Local\Temp\tmpo4xehaa8\ipykernel_18296\1962184903.py:28: RuntimeWarning: invalid value encountered in divide
  out[:] = pd.Series(volume[-1] / adv20[-1]).rank(pct=True)


2016-01-12 | Longs 34 | Shorts 38 | 78331.86508906682
2016-01-13 | Longs 33 | Shorts 38 | 71473.48358726881
2016-01-14 | Longs 36 | Shorts 40 | 76019.25706102916
2016-01-15 | Longs 31 | Shorts 46 | 71186.53480580031
2016-01-19 | Longs 31 | Shorts 45 | 67903.69152985961
2016-01-20 | Longs 37 | Shorts 38 | 67121.49052720444
2016-01-21 | Longs 35 | Shorts 42 | 71160.97952593688
2016-01-22 | Longs 30 | Shorts 49 | 74345.75375133568
2016-01-25 | Longs 22 | Shorts 56 | 69483.60798940247
2016-01-26 | Longs 28 | Shorts 50 | 72753.1176156093
2016-01-27 | Longs 32 | Shorts 43 | 72232.20182342757
2016-01-28 | Longs 33 | Shorts 44 | 72447.7988197593
2016-01-29 | Longs 35 | Shorts 41 | 78567.29991779634
2016-02-01 | Longs 26 | Shorts 47 | 78004.11439614024
2016-02-02 | Longs 25 | Shorts 49 | 71704.03725645252
2016-02-03 | Longs 28 | Shorts 47 | 74691.03041628691
2016-02-04 | Longs 25 | Shorts 50 | 76562.72313493748
2016-02-05 | Longs 22 | Shorts 54 | 69588.83077463726
2016-02-08 | Longs 25 | Shorts

C:\Users\nickt\AppData\Local\Temp\tmpo4xehaa8\ipykernel_18296\1962184903.py:28: RuntimeWarning: invalid value encountered in divide
  out[:] = pd.Series(volume[-1] / adv20[-1]).rank(pct=True)


: 